In [ ]:
import plotly.plotly as py
import plotly.tools as tls
from plotly.graph_objs import *
import numpy as np 
import datetime
import time


stream_id = 'i83gzt2cr7'
stream = Stream(
    token=stream_id,
    maxpoints=80
)

trace1 = Scatter(
    x=[],
    y=[],
    mode='lines+markers',
    stream=stream         # (!) embed stream id, 1 per trace
)

data = Data([trace1])

layout = Layout(title='FPGA feedback')
fig = Figure(data=data, layout=layout)
unique_url = py.plot(fig, filename='s7_first-stream')

s = py.Stream(stream_id)
s.open()



i = 0    # a counter
k = 5    # some shape parameter
N = 2000  # number of points to be plotted

# Delay start of stream by 5 sec (time to switch tabs)
time.sleep(5)

while True:
    x = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
    y = (np.cos(k*i/50.)*np.cos(i/50.)+np.random.randn(1))[0]
    s.write(dict(x=x, y=y))
    time.sleep(0.08)

s.close()

In [5]:
file_object = open("58_GHz_antenna_bot_4mm.stl.gcode", 'r')

In [6]:
file_object

<_io.TextIOWrapper name='58_GHz_antenna_bot_4mm.stl.gcode' mode='r' encoding='cp1252'>

In [8]:
print(file_object.read())

;FLAVOR:UltiGCode
;TIME:6666
;MATERIAL:6666
;MATERIAL2:0
;NOZZLE_DIAMETER:0.4

;Generated with Cura_SteamEngine master

;LAYER_COUNT:14
;LAYER:0
M107
G10
G0 F7200.000000 X41.373 Y41.124 Z0.300
;TYPE:SKIRT
G11
G1 F1800.000 X41.729 Y40.779 E0.05949
G1 X42.617 Y39.972 E0.20348
G1 X42.999 Y39.646 E0.26374
G1 X43.938 Y38.897 E0.40788
G1 X44.341 Y38.596 E0.46824
G1 X45.325 Y37.910 E0.61218
G1 X45.745 Y37.637 E0.67229
G1 X46.772 Y37.015 E0.81637
G1 X47.206 Y36.771 E0.87612
G1 X48.270 Y36.216 E1.02013
G1 X48.718 Y36.001 E1.07976
G1 X49.815 Y35.515 E1.22374
G1 X50.279 Y35.326 E1.28386
G1 X51.406 Y34.911 E1.42798
G1 X51.884 Y34.753 E1.48839
G1 X53.034 Y34.412 E1.63233
G1 X53.517 Y34.285 E1.69226
G1 X54.688 Y34.018 E1.83638
G1 X55.173 Y33.924 E1.89567
G1 X56.357 Y33.732 E2.03960
G1 X56.859 Y33.667 E2.10035
G1 X58.054 Y33.552 E2.24441
G1 X58.460 Y33.523 E2.29325
G1 X59.259 Y33.489 E2.38922
G1 X59.772 Y33.474 E2.45081
G1 X60.562 Y33.491 E2.54563
G1 X61.345 Y33.588 E2.64031
G1 X62.115 Y33.763 E2.735

In [3]:
import re
import plotly.plotly as py
import plotly.graph_objs as go
import time

#Extract G-codes from File
with open("58_GHz_antenna_bot_4mm.stl.gcode", 'r') as input_file:
    unclean_entries = input_file.readlines()
gcodes_sequence = []
for unclean_entry in unclean_entries:
    if 'G0 ' in unclean_entry or 'G1 ' in unclean_entry:
        gcodes_sequence.append(unclean_entry)

#Extract X Y Z points
segment_points = []
x_points = []
y_points =[]
z_points = []
z = 0
for gcode in gcodes_sequence:
    x_match = re.search('X(\d*\.\d*)', gcode)
    y_match = re.search('Y(\d*\.\d*)', gcode)
    z_match = re.search('Z(\d*\.\d*)', gcode)
    x = float(x_match.group(1))
    x_points.append(x)
    y = float(y_match.group(1))
    y_points.append(y)
    if z_match:
        z = float(z_match.group(1))
    z_points.append(z)
    xyz = (x, y, z)
    segment_points.append(xyz)

#print(segment_points)
#print(x_points)
#print(y_points)
#print(z_points)

#Streaming to a plot for dynamic update
stream_id = '3syvwhg0c9'
stream_1 = dict(
    token=stream_id,
    maxpoints=80
)

trace1 = go.Scatter3d(
    x = x_points,
    y = y_points,
    z = z_points[0:len(z_points)],
    mode = 'markers',
    marker= dict(
        size = 2,
        line = dict(
            color = 'rgba(217, 217, 217, 0.14)',
            width =0.3
        ),
        opacity = 0.8
    ),
    stream = stream_1
)

data = go.Data([trace1])
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)




fig = go.Figure(data=data, layout=layout)
url = py.plot(fig, filename='simple-3d-scatter')

s = py.Stream(stream_id)
s.open()

for i, val in enumerate(x_points):
    s.write(dict(x=x_points[i], y=y_points[i]))
    time.sleep(0.08)

s.close()